# Add Solar Angles to PV Dataset

In [16]:
import re
import tensorflow as tf
import pandas as pd
import numpy as np
import tarfile
from pathlib import Path

## 1. Load Dataset

In [4]:
csv_path = tf.keras.utils.get_file(
    origin='https://itsci.mju.ac.th/downloads/watcharin/datasets/pv/cmmju_15min_filled.csv.tar.gz'
)
csv_path

'C:\\Users\\ASUS\\.keras\\datasets\\cmmju_15min_filled.csv.tar.gz'

In [7]:
df = pd.read_csv(csv_path, index_col='datetime', parse_dates=True)
df.head(3)

,ambient_temp,current_power,value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption,module_temp,total_irradiation,cc,...,sp,sshf,ssr,ssrd,str,strd,t2m,tp,u10,v10
datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-06 11:15:00,30.288889,46519.355556,37280.422222,0.0,9238.933333,37280.422222,37280.422222,58.022222,798.444444,0.000002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-06 11:30:00,30.076923,47600.714286,39472.626374,0.0,9397.756098,39132.406593,39132.406593,58.263736,812.571429,0.000002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-06 11:45:00,30.629213,47056.988764,32669.393258,0.0,14387.595506,32669.393258,32669.393258,57.235955,807.224719,0.000001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2. Calculate Solar Angles

In [8]:
# Parameters for Chiang Mai
lat_point = 18.899741434351892
lon_point = 99.01248957594561
tz_offset_hours = 7

dt_index = df.index

# Calculations from the reference notebook
phi = np.deg2rad(lat_point)
doy = dt_index.dayofyear.values.astype(float)
local_minutes = (dt_index.hour.values * 60 + dt_index.minute.values).astype(float)

gamma = (2.0 * np.pi / 365.0) * (doy - 1.0 + (local_minutes - 720.0) / 1440.0)

EoT = 229.18 * (
    0.000075
    + 0.001868 * np.cos(gamma)
    - 0.032077 * np.sin(gamma)
    - 0.014615 * np.cos(2.0 * gamma)
    - 0.040849 * np.sin(2.0 * gamma)
)

delta = (
    0.006918
    - 0.399912 * np.cos(gamma)
    + 0.070257 * np.sin(gamma)
    - 0.006758 * np.cos(2.0 * gamma)
    + 0.000907 * np.sin(2.0 * gamma)
    - 0.002697 * np.cos(3.0 * gamma)
    + 0.00148  * np.sin(3.0 * gamma)
)

time_offset = EoT + 4.0 * lon_point - 60.0 * tz_offset_hours
solar_minutes = local_minutes + time_offset
H_deg = solar_minutes / 4.0 - 180.0
H_rad = np.deg2rad(H_deg)

cos_zenith = np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.cos(H_rad)
cos_zenith = np.clip(cos_zenith, -1.0, 1.0)
zenith_deg = np.degrees(np.arccos(cos_zenith))
elevation_deg = 90.0 - zenith_deg

azimuth_deg = (
    np.degrees(np.arctan2(np.sin(H_rad), np.cos(H_rad) * np.sin(phi) - np.tan(delta) * np.cos(phi)))
    + 180.0
)
azimuth_deg = np.mod(azimuth_deg, 360.0)

# Incidence Angle (assuming horizontal surface)
tilt_surface_deg = 0.0
azimuth_surface_deg = 180.0
beta = np.deg2rad(tilt_surface_deg)
psi_s = np.deg2rad(azimuth_surface_deg)
alpha_rad = np.deg2rad(elevation_deg)
psi_rad = np.deg2rad(azimuth_deg)
cos_incidence = np.sin(alpha_rad) * np.cos(beta) + np.cos(alpha_rad) * np.sin(beta) * np.cos(psi_rad - psi_s)
cos_incidence = np.clip(cos_incidence, -1.0, 1.0)
incidence_deg = np.degrees(np.arccos(cos_incidence))
incidence_deg = np.where(elevation_deg > 0.0, incidence_deg, np.nan)

## 3. Add Features to DataFrame

In [9]:
df['equation_of_time_min'] = EoT
df['zenith_deg'] = zenith_deg
df['elevation_deg'] = elevation_deg
df['azimuth_deg'] = azimuth_deg
df['declination_deg'] = np.degrees(delta)
df['hour_angle_deg'] = H_deg
df['incidence_deg'] = incidence_deg

## 4. Display Result

In [11]:
print(df.shape)
print(df.columns)
df.head()

(138238, 34)
Index(['ambient_temp', 'current_power', 'value_of_consumption',
       'external_energy_supply', 'grid_feed_in', 'internal_power_supply',
       'self_consumption', 'module_temp', 'total_irradiation', 'cc', 'q', 'r',
       't', 'u', 'v', 'fal', 'slhf', 'sp', 'sshf', 'ssr', 'ssrd', 'str',
       'strd', 't2m', 'tp', 'u10', 'v10', 'equation_of_time_min', 'zenith_deg',
       'elevation_deg', 'azimuth_deg', 'declination_deg', 'hour_angle_deg',
       'incidence_deg'],
      dtype='object')


,ambient_temp,current_power,value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption,module_temp,total_irradiation,cc,...,tp,u10,v10,equation_of_time_min,zenith_deg,elevation_deg,azimuth_deg,declination_deg,hour_angle_deg,incidence_deg
datetime,,,,,,,,,,,,,,,,,,,,,
2021-11-06 11:15:00,30.288889,46519.355556,37280.422222,0.0,9238.933333,37280.422222,37280.422222,58.022222,798.444444,2.297362e-06,...,0.0,0.0,0.0,16.244239,36.998137,53.001863,158.619243,-15.751553,-13.176451,36.998137
2021-11-06 11:30:00,30.076923,47600.714286,39472.626374,0.0,9397.756098,39132.406593,39132.406593,58.263736,812.571429,2.376975e-06,...,0.0,0.0,0.0,16.243603,35.874705,54.125295,164.395912,-15.754719,-9.426610,35.874705
2021-11-06 11:45:00,30.629213,47056.988764,32669.393258,0.0,14387.595506,32669.393258,32669.393258,57.235955,807.224719,1.268101e-06,...,0.0,0.0,0.0,16.242965,35.105017,54.894983,170.471336,-15.757885,-5.676769,35.105017
2021-11-06 12:00:00,31.021978,48415.835165,36126.516484,0.0,12289.318681,36126.516484,36126.516484,56.846154,817.527473,0.000000e+00,...,0.0,0.0,0.0,16.242326,34.712629,55.287371,176.742301,-15.761050,-1.926929,34.712629
2021-11-06 12:15:00,31.831461,49345.404494,36879.966292,0.0,12465.438202,36879.966292,36879.966292,55.842697,816.651685,-6.155763e-07,...,0.0,0.0,0.0,16.241685,34.710348,55.289652,183.081860,-15.764215,1.822911,34.710348


In [17]:
filename = Path(csv_path).name
base = re.split(r'\.', filename, maxsplit=1)[0]
print(base)

cmmju_15min_filled


In [19]:
# Save
df.to_csv('C:\\Users\\ASUS\\.keras\\datasets\\' + base + '_extra.csv')